In [8]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ReLU, BatchNormalization, Dropout
from tensorflow.keras import optimizers
import numpy as np
import os
import sys
from six.moves import cPickle
import tensorflow as tf
import matplotlib.pyplot as plt

# hyper parameter
batch_size = 32
num_classes = 100
epochs = 30

# 데이터 총 개수 : 40000개
# 클래스 개수 : 100개, 클래스 별 400개 이미지

steps_per_epoch = int(40000/batch_size)
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'trained_model.h5'

In [2]:
import sklearn.model_selection as sk

def load_data():
    path = './data'

    fpath = os.path.join(path, 'train_data')
    
    with open(fpath, 'rb') as f:
        d = cPickle.load(f, encoding='bytes')
        
    X_train = d['data']
    y_train = d['labels']
    y_train = np.reshape(y_train, (len(y_train), 1))

    return X_train, y_train

x_train, y_train = load_data()

# x_train : total img dataset
# y_train : total label datset


# test, train dataset 분류하기 
X_train, X_test, y_train, y_test = sk.train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.33,
                                                    random_state=42,
                                                    shuffle=True)


In [3]:
# normalize image data

print ("mean before normalization:", np.mean(X_train)) 
print ("std before normalization:", np.std(X_train))

mean=[0,0,0]
std=[0,0,0]

newX_train = np.ones(X_train.shape)
newX_test = np.ones(X_test.shape)

for i in range(3):
    mean[i] = np.mean(X_train[:,:,:,i])
    std[i] = np.std(X_train[:,:,:,i])
       
for i in range(3):
    newX_train[:,:,:,i] = X_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]
    newX_test[:,:,:,i] = X_test[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]

X_train = newX_train
X_test = newX_test

print ("mean after normalization:", np.mean(X_train))
print ("std after normalization:", np.std(X_train))
print(X_train.max())

mean before normalization: 121.98364992663635
std before normalization: 68.30227829343998
mean after normalization: 5.568237965794131e-17
std after normalization: 0.9999999999999987
2.025461104398623


In [4]:
# label preprocessing: one hot encoding

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
# make dataset

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_dataset = val_dataset.batch(32)

In [13]:
# model5
# add layer

from keras.optimizers import adam

model5 = Sequential()

model5.add(Conv2D(50, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size = (2,2)))

model5.add(Conv2D(100, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size = (2,2)))

model5.add(Conv2D(200, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size = (2,2)))


model5.add(Conv2D(400, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model5.add(BatchNormalization())
model5.add(Activation('relu'))
model5.add(MaxPooling2D(pool_size = (2,2)))

model5.add(Flatten())

model5.add(Dense(800))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))

adam = optimizers.Adam(lr = 0.001)

model5.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

model5.fit(dataset, 
           epochs=epochs,
           shuffle=True,
           validation_data=val_dataset,
           callbacks=callbacks,
           workers=4)



Epoch 1/30
838/838 [==============================] - 73s 87ms/step - loss: 4.7776 - accuracy: 0.0733 - val_loss: 3.5897 - val_accuracy: 0.1436
Epoch 2/30
838/838 [==============================] - 72s 86ms/step - loss: 3.4288 - accuracy: 0.1769 - val_loss: 3.1066 - val_accuracy: 0.2323
Epoch 3/30
838/838 [==============================] - 74s 89ms/step - loss: 2.9113 - accuracy: 0.2734 - val_loss: 2.7291 - val_accuracy: 0.3092
Epoch 4/30
838/838 [==============================] - 79s 94ms/step - loss: 2.5167 - accuracy: 0.3517 - val_loss: 2.5472 - val_accuracy: 0.3540
Epoch 5/30
838/838 [==============================] - 77s 92ms/step - loss: 2.1739 - accuracy: 0.4278 - val_loss: 2.5278 - val_accuracy: 0.3672
Epoch 6/30
838/838 [==============================] - 78s 93ms/step - loss: 1.8481 - accuracy: 0.4990 - val_loss: 2.5041 - val_accuracy: 0.3807
Epoch 7/30
838/838 [==============================] - 77s 92ms/step - loss: 1.5219 - accuracy: 0.5746 - val_loss: 2.6441 - val_accuracy:

In [12]:
from tensorflow.keras.layers import LeakyReLU

# model6
# relu >> leakyrelu

from keras.optimizers import adam

model6 = Sequential()

model6.add(Conv2D(50, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model6.add(BatchNormalization())
model6.add(LeakyReLU(alpha=0.05))
model6.add(MaxPooling2D(pool_size = (2,2)))

model6.add(Conv2D(100, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model6.add(BatchNormalization())
model6.add(LeakyReLU(alpha=0.05))
model6.add(MaxPooling2D(pool_size = (2,2)))

model6.add(Conv2D(200, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model6.add(BatchNormalization())
model6.add(LeakyReLU(alpha=0.05))
model6.add(MaxPooling2D(pool_size = (2,2)))


model6.add(Conv2D(400, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model6.add(BatchNormalization())
model6.add(LeakyReLU(alpha=0.05))
model6.add(MaxPooling2D(pool_size = (2,2)))

model6.add(Flatten())

model6.add(Dense(800))
model6.add(Dropout(0.5))
model6.add(Dense(num_classes, activation='softmax'))

adam = optimizers.Adam(lr = 0.001)

model6.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

model6.fit(dataset, 
           epochs=epochs,
           shuffle=True,
           validation_data=val_dataset,
           callbacks=callbacks,
           workers=4)


# 오 .. relu 보단 leakyrelu 가 약간 더 성능이 좋다. (학습 시간은 좀 더 걸리지만.. )

Epoch 1/30
838/838 [==============================] - 80s 96ms/step - loss: 4.7668 - accuracy: 0.0703 - val_loss: 3.6346 - val_accuracy: 0.1378
Epoch 2/30
838/838 [==============================] - 83s 99ms/step - loss: 3.4288 - accuracy: 0.1787 - val_loss: 3.0767 - val_accuracy: 0.2330
Epoch 3/30
838/838 [==============================] - 78s 93ms/step - loss: 2.8877 - accuracy: 0.2785 - val_loss: 2.6744 - val_accuracy: 0.3133
Epoch 4/30
838/838 [==============================] - 81s 96ms/step - loss: 2.4930 - accuracy: 0.3579 - val_loss: 2.5854 - val_accuracy: 0.3374
Epoch 5/30
838/838 [==============================] - 80s 96ms/step - loss: 2.1574 - accuracy: 0.4287 - val_loss: 2.5072 - val_accuracy: 0.3706
Epoch 6/30
838/838 [==============================] - 81s 97ms/step - loss: 1.8268 - accuracy: 0.5043 - val_loss: 2.5034 - val_accuracy: 0.3905
Epoch 7/30
838/838 [==============================] - 79s 95ms/step - loss: 1.5045 - accuracy: 0.5810 - val_loss: 2.6686 - val_accuracy:

In [21]:
from tensorflow.keras.layers import LeakyReLU

# model7
# optimize parameter : alpha 0.05 -> 0.01

from keras.optimizers import adam

model7 = Sequential()

model7.add(Conv2D(50, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model7.add(BatchNormalization())
model7.add(LeakyReLU(alpha=0.01))
model7.add(MaxPooling2D(pool_size = (2,2)))

model7.add(Conv2D(100, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model7.add(BatchNormalization())
model7.add(LeakyReLU(alpha=0.01))
model7.add(MaxPooling2D(pool_size = (2,2)))

model7.add(Conv2D(200, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model7.add(BatchNormalization())
model7.add(LeakyReLU(alpha=0.01))
model7.add(MaxPooling2D(pool_size = (2,2)))


model7.add(Conv2D(400, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model7.add(BatchNormalization())
model7.add(LeakyReLU(alpha=0.01))
model7.add(MaxPooling2D(pool_size = (2,2)))

model7.add(Flatten())

model7.add(Dense(800))
model7.add(Dropout(0.5))
model7.add(Dense(num_classes, activation='softmax'))

adam = optimizers.Adam(lr = 0.001)

model7.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

model7.fit(dataset, 
           epochs=epochs,
           shuffle=True,
           validation_data=val_dataset,
           callbacks=callbacks,
           workers=4)


# alpha 값은 0.01 이 좋은 것 같다. 정확도가 많이 떨어진다.

Epoch 1/30
838/838 [==============================] - 81s 96ms/step - loss: 4.6828 - accuracy: 0.0696 - val_loss: 3.6109 - val_accuracy: 0.1322
Epoch 2/30
838/838 [==============================] - 80s 96ms/step - loss: 3.4560 - accuracy: 0.1718 - val_loss: 3.0518 - val_accuracy: 0.2308
Epoch 3/30
838/838 [==============================] - 76s 91ms/step - loss: 2.9543 - accuracy: 0.2657 - val_loss: 2.7722 - val_accuracy: 0.2899
Epoch 4/30
838/838 [==============================] - 87s 104ms/step - loss: 2.5595 - accuracy: 0.3430 - val_loss: 2.5989 - val_accuracy: 0.3291
Epoch 5/30
838/838 [==============================] - 86s 103ms/step - loss: 2.2213 - accuracy: 0.4173 - val_loss: 2.5527 - val_accuracy: 0.3558
Epoch 6/30
838/838 [==============================] - 85s 102ms/step - loss: 1.8938 - accuracy: 0.4858 - val_loss: 2.5803 - val_accuracy: 0.3673
Epoch 7/30
838/838 [==============================] - 88s 105ms/step - loss: 1.5743 - accuracy: 0.5599 - val_loss: 2.7526 - val_accur

In [23]:
from tensorflow.keras.layers import LeakyReLU

# model8
# add dropout

from keras.optimizers import adam

model8 = Sequential()

model8.add(Conv2D(50, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model8.add(BatchNormalization())
model8.add(LeakyReLU(alpha=0.05))
model8.add(MaxPooling2D(pool_size = (2,2)))

model8.add(Conv2D(100, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model8.add(BatchNormalization())
model8.add(LeakyReLU(alpha=0.05))
model8.add(MaxPooling2D(pool_size = (2,2)))
model8.add(Dropout(0.1))

model8.add(Conv2D(200, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model8.add(BatchNormalization())
model8.add(LeakyReLU(alpha=0.05))
model8.add(MaxPooling2D(pool_size = (2,2)))


model8.add(Conv2D(400, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model8.add(BatchNormalization())
model8.add(LeakyReLU(alpha=0.05))
model8.add(MaxPooling2D(pool_size = (2,2)))
model8.add(Dropout(0.1))

model8.add(Flatten())

model8.add(Dense(800))
model8.add(Dropout(0.5))
model8.add(Dense(num_classes, activation='softmax'))

adam = optimizers.Adam(lr = 0.001)

model8.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

model8.fit(dataset, 
           epochs=epochs,
           shuffle=True,
           validation_data=val_dataset,
           callbacks=callbacks,
           workers=4)


# 미약하게 정확도는 올라갔으나 test accuracy 도 매우 적게 올라가기 시작했다 


Epoch 1/30
838/838 [==============================] - 81s 97ms/step - loss: 4.6602 - accuracy: 0.0701 - val_loss: 3.6365 - val_accuracy: 0.1404
Epoch 2/30
838/838 [==============================] - 84s 100ms/step - loss: 3.4743 - accuracy: 0.1685 - val_loss: 3.1461 - val_accuracy: 0.2189
Epoch 3/30
838/838 [==============================] - 89s 107ms/step - loss: 2.9971 - accuracy: 0.2565 - val_loss: 2.7865 - val_accuracy: 0.2932
Epoch 4/30
838/838 [==============================] - 91s 109ms/step - loss: 2.6507 - accuracy: 0.3249 - val_loss: 2.5547 - val_accuracy: 0.3340
Epoch 5/30
838/838 [==============================] - 90s 108ms/step - loss: 2.3786 - accuracy: 0.3826 - val_loss: 2.4948 - val_accuracy: 0.3487
Epoch 6/30
838/838 [==============================] - 93s 111ms/step - loss: 2.1355 - accuracy: 0.4352 - val_loss: 2.3566 - val_accuracy: 0.3861
Epoch 7/30
838/838 [==============================] - 94s 112ms/step - loss: 1.9057 - accuracy: 0.4868 - val_loss: 2.3612 - val_acc

In [25]:
from tensorflow.keras.layers import LeakyReLU

# model9
# relu >> leakyrelu

from keras.optimizers import adam
from keras.regularizers import l2

model9 = Sequential()

model9.add(Conv2D(50, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model9.add(BatchNormalization())
model9.add(LeakyReLU(alpha=0.05))
model9.add(MaxPooling2D(pool_size = (2,2)))

model9.add(Conv2D(100, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model9.add(BatchNormalization())
model9.add(LeakyReLU(alpha=0.05))
model9.add(MaxPooling2D(pool_size = (2,2)))

model9.add(Conv2D(200, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model9.add(BatchNormalization())
model9.add(LeakyReLU(alpha=0.05))
model9.add(MaxPooling2D(pool_size = (2,2)))


model9.add(Conv2D(400, (3, 3), 
                  padding='same',
                  input_shape=X_train.shape[1:],))
model9.add(BatchNormalization())
model9.add(LeakyReLU(alpha=0.05))
model9.add(MaxPooling2D(pool_size = (2,2)))

model9.add(Flatten())

model9.add(Dense(800))
model9.add(Dropout(0.5))
model9.add(Dense(num_classes, activation='softmax'))

adam = optimizers.Adam(lr = 0.001)

model9.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

callbacks = [
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')
]

model9.fit(dataset, 
           epochs=epochs,
           shuffle=True,
           validation_data=val_dataset,
           callbacks=callbacks,
           workers=4)


# 오 .. relu 보단 leakyrelu 가 약간 더 성능이 좋다. (학습 시간은 좀 더 걸리지만.. )

Epoch 1/30
838/838 [==============================] - 82s 97ms/step - loss: 7.6274 - accuracy: 0.0580 - val_loss: 4.1734 - val_accuracy: 0.1002
Epoch 2/30
838/838 [==============================] - 82s 97ms/step - loss: 3.9688 - accuracy: 0.1298 - val_loss: 3.7862 - val_accuracy: 0.1586
Epoch 3/30
838/838 [==============================] - 83s 99ms/step - loss: 3.5824 - accuracy: 0.2041 - val_loss: 3.3973 - val_accuracy: 0.2433
Epoch 4/30
838/838 [==============================] - 91s 108ms/step - loss: 3.2986 - accuracy: 0.2653 - val_loss: 3.2359 - val_accuracy: 0.2774
Epoch 5/30
838/838 [==============================] - ETA: 0s - loss: 3.0823 - accuracy: 0.3212

KeyboardInterrupt: 